In [96]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LayerNormalization, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
from numpy.random import seed

In [97]:
Xl = pd.read_csv('Xl.csv',header=None)
Xs = pd.read_csv('Xs.csv',header=None)
R = pd.read_csv('y.csv',header = None)

In [98]:
y_agg=R.iloc[::-1].rolling(window=12).sum().iloc[::-1]

In [99]:
t_train_start = list(range(46*12))
t_train_end =[x+120 for x in t_train_start]
t_val_start= [x for x in t_train_end]
t_val_end = [x+60 for x in t_val_start]
t_test_start = [x for x in t_val_end]
t_test_end = [x for x in t_test_start]

In [100]:
np.shape(t_test_end) #model needed to be retrained every month

(552,)

In [101]:
t_test_end[551]

731

In [105]:
loss = []
preds = []
seed(1)
for retrain_idx in range(552):
    X = Xl.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Xl.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],X_idx]
    X_test = Xl.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],X_idx]
    y = y_agg.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    y_val = y_agg.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],:]
    y_test = y_agg.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],:]
    model = Sequential()
    model.add(LayerNormalization())
    model.add(Dense(128,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(64,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(32,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(16,activation='relu'))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(8,activation='relu'))
    model.add(Dense(1))
    optimizer = Adam(learning_rate=0.005)
    model.compile(optimizer=optimizer,loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', verbose=0, patience=3)
    history = model.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=64, epochs=100,verbose=1,callbacks=[early_stop])
    preds.append(model.predict(X_test))
    loss.append(min(history.history['val_loss']))


Epoch 1/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

2/2 [==============================] - 0s 103ms/step - loss: 0.3703 - val_loss: 0.0443
Epoch 2/100
2/2 [==============================] - 0s 16ms/step - loss: 0.0481 - val_loss: 0.0373
Epoch 3/100
2/2 [==============================] - 0s 18ms/step - loss: 0.0378 - val_loss: 0.0376
Epoch 4/100
2/2 [==============================] - 0s 15ms/step - loss: 0.0255 - val_loss: 0.0379
Epoch 5/100
2/2 [==============================] - 0s 15ms/step - loss: 0.0184 - val_loss: 0.0376
Epoch 1/100

If y

In [106]:
y_org = y_agg.loc[t_test_start[0]:t_test_end[551],:]

### Calculate R^2

In [107]:
1-np.sum((np.squeeze(y_org)-np.squeeze(preds))**2)/np.sum((y_org-np.mean(y_org))**2)

0   -0.489419
dtype: float64

### A good R^2 is expected to be between 0 to 0.05.